In [1]:
%pip install -U assemblyai 



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pyaudio


  Using cached PyAudio-0.2.13-cp311-cp311-macosx_11_0_arm64.whl
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install assemblyai\[extras\]


Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install tokenizers

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install langchain openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 7.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.4/425.4 kB 6.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 5.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import assemblyai as aai
from langchain.schema import (
    SystemMessage
)
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.schema import SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory

from transformers import AutoTokenizer, AutoModelForSequenceClassification


import torch
import re
from IPython.display import display, Markdown



aai.settings.api_key = "<api key>"


/Users/carlogiuseppesergi/anaconda3/envs/interpreter/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = AutoTokenizer.from_pretrained("mrsinghania/asr-question-detection")

classificator = AutoModelForSequenceClassification.from_pretrained("mrsinghania/asr-question-detection")

In [4]:
# Utilizza il tokenizer per preparare i dati per il modello

def predict_question(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = classificator(**inputs)
    predictions = outputs[0]
    probabilities = torch.nn.functional.softmax(predictions, dim=-1)
    predicted_class = torch.argmax(probabilities)
    return 'SENTENCE' if predicted_class == 0 else 'QUESTION'



In [5]:
print(predict_question('Hello'))
print(predict_question('How are you?'))
print(predict_question(''))


SENTENCE
QUESTION
SENTENCE


In [6]:
#seleziona solo domande
model = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')
#model = Ollama(model="mistral")

template = '''
You are a job interview assistan AI, you help the human with the job question, 
you are expert on IT and Software engineer and you should answer in very short way with a list, with the keys point.

EXAMPLE 1
Human: Could you explain me the SOLID principle?

IA:
Sure, I believe you're referring to the SOLID principles of object-oriented programming. Here they are:

1. **S - Single Responsibility Principle (SRP):**
   - A class should have only one reason to change. It means a class should only have one job or responsibility.

2. **O - Open/Closed Principle (OCP):**
   - Software entities (classes, modules, functions, etc.) should be open for extension, but closed for modification. This means that a class should be easily extendable without modifying the class itself.

3. **L - Liskov Substitution Principle (LSP):**
   - Subtypes must be substitutable for their base types. This means that if a program is using a base class, it should be able to use any of its subclasses without the program knowing it.

4. **I - Interface Segregation Principle (ISP):**
   - Clients should not be forced to depend upon interfaces they do not use. This means that a class should not have to implement methods it doesn't use.

5. **D - Dependency Inversion Principle (DIP):**
   - High-level modules should not depend on low-level modules. Both should depend on abstractions. This means that one should "depend upon abstractions, [not] concretions."

EXAMPLE 2

Human: What is React.js?

IA: 
Sure, here are the key points about React.js:

1. **JavaScript Library:** 
   - React.js is a JavaScript library developed by Facebook. It's used for building user interfaces, particularly for single-page applications.

2. **Component-Based:**
   - React allows developers to create large web applications which can change data, without reloading the page. The main purpose of React is to be fast, scalable, and simple.

3. **Virtual DOM:**
   - React creates an in-memory data structure cache, computes the resulting differences, and then updates the browser's displayed DOM efficiently. This allows a special feature called the 'Virtual DOM' that optimizes rendering and improves app performance.

4. **Unidirectional Data Flow:**
   - Data in React flows in a single direction which helps in maintaining control over it and helps in debugging and understanding the application.

5. **JSX:**
   - React uses JSX, a syntax extension for JavaScript, allowing mixing of HTML with JavaScript. This isn't required to use React, but it makes the code more readable and writeable.

6. **React Native:**
   - React can also render on the server using Node, and it can power native mobile apps using React Native, a separate platform that uses the principles of React to allow powerful native mobile app development.


'''

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=template), # The persistent system prompt
    MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
    HumanMessagePromptTemplate.from_template("{text}"), # Where the human input will injected
])
memory = ConversationSummaryBufferMemory(memory_key="chat_history", llm=OpenAI(), return_messages=True, max_token_limit=1000)



conversation = LLMChain(
    llm=model,
    prompt=prompt,
    memory=memory
)
def call_gpt(text):
 response = conversation.predict(text=text)
 return response
#risponde alle domande

In [7]:
response = call_gpt('What is a Database?')
print(response)

response = call_gpt('And the difference with Redis?')

print(response)

Sure, here are the key points about a database:

1. **Data Storage:** 
   - A database is a structured collection of data that is organized and stored for easy access, retrieval, and management.

2. **Data Management System:**
   - It is managed by a Database Management System (DBMS), which provides tools and functions to create, modify, and query the data.

3. **Structured Format:**
   - Data in a database is organized into tables, which consist of rows (records) and columns (fields). This structured format allows for efficient storage and retrieval of data.

4. **Data Integrity and Security:**
   - Databases ensure data integrity by enforcing constraints and rules on the data. They also provide security features to protect sensitive information.

5. **Data Manipulation:**
   - DBMS allows users to manipulate the data through operations like inserting, updating, deleting, and querying data using SQL (Structured Query Language).

6. **Scalability and Performance:**
   - Databases can h

In [8]:
def is_sentence_complete(sentence):
    # Cerca un carattere di punto, di punto esclamativo o di punto interrogativo alla fine della stringa
    if re.search(r'[.!?]$', sentence):
        return True
    else:
        return False

In [9]:
def process_transcript(transcript):
    if transcript.strip() and is_sentence_complete(transcript):
        # Split transcript into sentences based on punctuation marks.
        sentences = re.split(r'[.?!]', transcript)
        
        for sentence in sentences:
            sentence = sentence.strip()  # remove leading/trailing whitespace
            if sentence:  # to make sure sentence is not empty
                question = predict_question(sentence)
                if question == 'QUESTION':
                    response = call_gpt(sentence)
                    return response
      
print(process_transcript(' '))
print(process_transcript('What is a singleton class?'))

None
A singleton class is a design pattern that restricts the instantiation of a class to a single object. Here are the key points about singleton classes:

1. **Single Instance:**
   - A singleton class allows only one instance of the class to be created and provides a global point of access to that instance.

2. **Private Constructor:**
   - To enforce the single instance rule, the constructor of a singleton class is made private, preventing direct instantiation of the class from outside.

3. **Static Instance Method:**
   - The singleton class provides a static method that returns the single instance of the class. This method is typically named `getInstance()`.

4. **Lazy Initialization:**
   - The instance of the singleton class is created lazily, i.e., only when the `getInstance()` method is called for the first time. This ensures efficient resource utilization.

5. **Global Access:**
   - Once the singleton instance is created, it can be accessed globally from anywhere in the cod

In [12]:
subtitle_area = display("", display_id=True)
output_area = display("", display_id=True)
 
# create variable to hold history of messages
output_history = ""

def on_open(session_opened: aai.RealtimeSessionOpened):
  "This function is called when the connection has been established."
  print("Session ID:", session_opened.session_id)

def on_data(transcript: aai.RealtimeTranscript):
  "This function is called when a new transcript has been received."

  global subtitle_history, output_history

  if not transcript.text:
    return

  if isinstance(transcript, aai.RealtimeFinalTranscript):
    # accumulate the transcript text
    subtitle_area.update(transcript.text)
    response = process_transcript(transcript.text)
    if response != None:
      # accumulate the response
      output_history += "\n\n\n" + response
      output_area.update(Markdown(output_history))
  else:
    # accumulate the transcript text
    subtitle_area.update(transcript.text)
    # response = process_transcript(transcript.text)
    # if response != None:
    #   #accumulate the response
    #   output_history += "\n" + response
    #   output_area.update(Markdown(output_history))

def on_error(error: aai.RealtimeError):
  "This function is called when the connection has been closed."

  print("An error occured:", error)

def on_close():
  "This function is called when the connection has been closed."

  print("Closing Session")
  
transcriber = aai.RealtimeTranscriber(
  on_data=on_data,
  on_error=on_error,
  sample_rate=44_100,
  on_open=on_open, # optional
  on_close=on_close, # optional
)

# Start the connection
transcriber.connect()

# Open a microphone stream
microphone_stream = aai.extras.MicrophoneStream()

# Press CTRL+C to abort
transcriber.stream(microphone_stream)

transcriber.close()

'What is jsx.'




In React, the Virtual DOM (Document Object Model) is a lightweight copy of the actual DOM. It is a JavaScript representation of the HTML structure of your application. 

Here are the key points about the Virtual DOM in React:

1. **Efficient Rendering:** React uses the Virtual DOM to efficiently update and render components. When there is a change in the state or props of a component, React creates a new Virtual DOM tree and compares it with the previous one to identify the minimal set of changes needed to update the actual DOM.

2. **Optimized Updates:** By comparing the Virtual DOM trees, React determines the specific changes that need to be made to the actual DOM. It then applies these changes in a batch update, minimizing the number of DOM manipulations and improving performance.

3. **Faster Rendering:** The Virtual DOM allows React to perform updates in memory, rather than directly manipulating the actual DOM. This results in faster rendering and improved overall performance of the application.

4. **Reconciliation:** React's reconciliation algorithm efficiently updates the Virtual DOM and reconciles it with the actual DOM. It identifies the differences between the two and updates only the necessary parts, reducing the amount of work required for rendering.

5. **Developer-Friendly:** The Virtual DOM provides a simplified programming model for developers. They can work with a virtual representation of the DOM, which is easier to manipulate and reason about, without directly interacting with the actual DOM.

Overall, the Virtual DOM in React helps optimize rendering, improve performance, and simplify the development process by providing an efficient and developer-friendly way to update and manipulate the DOM.


I'm sorry, but I'm not familiar with the term "jj" in the context of IT or software engineering. It could be an abbreviation or acronym specific to a certain domain or technology. If you can provide more context or clarify what "jj" refers to, I'll do my best to assist you.


JSX stands for JavaScript XML. It is a syntax extension for JavaScript that allows you to write HTML-like code within JavaScript. JSX is primarily used in React to define the structure and appearance of components.

Here are the key points about JSX:

1. **HTML-like Syntax:** JSX resembles HTML syntax, allowing you to write familiar HTML tags and attributes directly in your JavaScript code.

2. **Combining JavaScript and HTML:** JSX allows you to embed JavaScript expressions within curly braces {} directly in the HTML-like code. This enables dynamic rendering and manipulation of data.

3. **Component-Based Structure:** JSX is used to define the structure and appearance of React components. Each JSX element represents a React component or HTML element.

4. **Transpilation:** JSX code needs to be transpiled into regular JavaScript code before it can be executed in the browser. Tools like Babel are commonly used to convert JSX into JavaScript that the browser can understand.

5. **Readability and Maintainability:** JSX improves code readability by providing a declarative and intuitive way to describe the UI structure. It also helps maintain a clear separation between the UI and logic.

Overall, JSX is a powerful feature in React that allows developers to write HTML-like code within JavaScript, making it easier to build and manage complex user interfaces.

Session ID: d42ae2b0-ce8f-4a5f-93f3-d58f9a27993e
Closing Session
